In [4]:
import os
import random
from collections import defaultdict
from enum import Enum
from typing import Tuple, List

import numpy as np
import torch
from PIL import Image
from torch.utils.data import Dataset, DataLoader, Subset, random_split
from torchvision.transforms import Resize, ToTensor, Normalize, Compose, CenterCrop, ColorJitter
import torchvision.models as models


import torch
from torchvision import models
import torch.nn as nn
import torch.nn.functional as F

In [4]:
IMG_EXTENSIONS = [
    ".jpg", ".JPG", ".jpeg", ".JPEG", ".png",
    ".PNG", ".ppm", ".PPM", ".bmp", ".BMP",
]


def is_image_file(filename):
    return any(filename.endswith(extension) for extension in IMG_EXTENSIONS)


class BaseAugmentation:
    def __init__(self, resize, mean, std, **args):
        self.transform = Compose([
            Resize(resize, Image.BILINEAR),
            ToTensor(),
            Normalize(mean=mean, std=std),
        ])

    def __call__(self, image):
        return self.transform(image)


class AddGaussianNoise(object):
    """
        transform 에 없는 기능들은 이런식으로 __init__, __call__, __repr__ 부분을
        직접 구현하여 사용할 수 있습니다.
    """

    def __init__(self, mean=0., std=1.):
        self.std = std
        self.mean = mean

    def __call__(self, tensor):
        return tensor + torch.randn(tensor.size()) * self.std + self.mean

    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)


class CustomAugmentation:
    def __init__(self, resize, mean, std, **args):
        self.transform = Compose([
            CenterCrop((320, 256)),
            Resize(resize, Image.BILINEAR),
            ColorJitter(0.1, 0.1, 0.1, 0.1),
            ToTensor(),
            Normalize(mean=mean, std=std),
            AddGaussianNoise()
        ])

    def __call__(self, image):
        return self.transform(image)


class MaskLabels(int, Enum):
    MASK = 0
    INCORRECT = 1
    NORMAL = 2


class GenderLabels(int, Enum):
    MALE = 0
    FEMALE = 1

    @classmethod
    def from_str(cls, value: str) -> int:
        value = value.lower()
        if value == "male":
            return cls.MALE
        elif value == "female":
            return cls.FEMALE
        else:
            raise ValueError(f"Gender value should be either 'male' or 'female', {value}")


class AgeLabels(int, Enum):
    YOUNG = 0
    MIDDLE = 1
    OLD = 2

    @classmethod
    def from_number(cls, value: str) -> int:
        try:
            value = int(value)
        except Exception:
            raise ValueError(f"Age value should be numeric, {value}")

        if value < 30:
            return cls.YOUNG
        elif value < 60:
            return cls.MIDDLE
        else:
            return cls.OLD


In [5]:
class MaskBaseDataset(Dataset):
    num_classes = 3 * 2 * 3

    _file_names = {
        "mask1": MaskLabels.MASK,
        "mask2": MaskLabels.MASK,
        "mask3": MaskLabels.MASK,
        "mask4": MaskLabels.MASK,
        "mask5": MaskLabels.MASK,
        "incorrect_mask": MaskLabels.INCORRECT,
        "normal": MaskLabels.NORMAL
    }

    # image_paths = []
    # mask_labels = []
    # gender_labels = []
    # age_labels = []

    def __init__(self, data_dir, mean=(0.548, 0.504, 0.479), std=(0.237, 0.247, 0.246), val_ratio=0.2):
        self.data_dir = data_dir
        self.mean = mean
        self.std = std
        self.val_ratio = val_ratio

        self.image_paths = []
        self.mask_labels = []
        self.gender_labels = []
        self.age_labels = []

        self.transform = None
        self.setup()
        self.calc_statistics()



    def setup(self):
        profiles = os.listdir(self.data_dir)
        for profile in profiles:
            if profile.startswith("."):  # "." 로 시작하는 파일은 무시합니다
                continue

            img_folder = os.path.join(self.data_dir, profile)
            for file_name in os.listdir(img_folder):
                _file_name, ext = os.path.splitext(file_name)
                if _file_name not in self._file_names:  # "." 로 시작하는 파일 및 invalid 한 파일들은 무시합니다
                    continue

                img_path = os.path.join(self.data_dir, profile, file_name)  # (resized_data, 000004_male_Asian_54, mask1.jpg)
                mask_label = self._file_names[_file_name]

                id, gender, race, age = profile.split("_")
                gender_label = GenderLabels.from_str(gender)
                age_label = AgeLabels.from_number(age)

                self.image_paths.append(img_path)
                self.mask_labels.append(mask_label)
                self.gender_labels.append(gender_label)
                self.age_labels.append(age_label)

    def calc_statistics(self):
        has_statistics = self.mean is not None and self.std is not None
        if not has_statistics:
            print("[Warning] Calculating statistics... It can take a long time depending on your CPU machine")
            sums = []
            squared = []
            for image_path in self.image_paths[:3000]:
                image = np.array(Image.open(image_path)).astype(np.int32)
                sums.append(image.mean(axis=(0, 1)))
                squared.append((image ** 2).mean(axis=(0, 1)))

            self.mean = np.mean(sums, axis=0) / 255
            self.std = (np.mean(squared, axis=0) - self.mean ** 2) ** 0.5 / 255

    def set_transform(self, transform):
        self.transform = transform

    def __getitem__(self, index):
        assert self.transform is not None, ".set_tranform 메소드를 이용하여 transform 을 주입해주세요"

        image = self.read_image(index)
        mask_label = self.get_mask_label(index)
        gender_label = self.get_gender_label(index)
        age_label = self.get_age_label(index)
        multi_class_label = self.encode_multi_class(mask_label, gender_label, age_label)

        image_transform = self.transform(image)
        return image_transform, multi_class_label

    def __len__(self):
        return len(self.image_paths)

    def get_mask_label(self, index) -> MaskLabels:
        return self.mask_labels[index]

    def get_gender_label(self, index) -> GenderLabels:
        return self.gender_labels[index]

    def get_age_label(self, index) -> AgeLabels:
        return self.age_labels[index]

    def read_image(self, index):
        image_path = self.image_paths[index]
        return Image.open(image_path)

    @staticmethod
    def encode_multi_class(mask_label, gender_label, age_label) -> int:
        return mask_label * 6 + gender_label * 3 + age_label

    @staticmethod
    def decode_multi_class(multi_class_label) -> Tuple[MaskLabels, GenderLabels, AgeLabels]:
        mask_label = (multi_class_label // 6) % 3
        gender_label = (multi_class_label // 3) % 2
        age_label = multi_class_label % 3
        return mask_label, gender_label, age_label

    @staticmethod
    def denormalize_image(image, mean, std):
        img_cp = image.copy()
        img_cp *= std
        img_cp += mean
        img_cp *= 255.0
        img_cp = np.clip(img_cp, 0, 255).astype(np.uint8)
        return img_cp

    def split_dataset(self) -> Tuple[Subset, Subset]:
        """
        데이터셋을 train 과 val 로 나눕니다,
        pytorch 내부의 torch.utils.data.random_split 함수를 사용하여
        torch.utils.data.Subset 클래스 둘로 나눕니다.
        구현이 어렵지 않으니 구글링 혹은 IDE (e.g. pycharm) 의 navigation 기능을 통해 코드를 한 번 읽어보는 것을 추천드립니다^^
        """
        n_val = int(len(self) * self.val_ratio)
        n_train = len(self) - n_val
        train_set, val_set = random_split(self, [n_train, n_val])
        return train_set, val_set


class MaskSplitByProfileDataset(MaskBaseDataset):
    """
        train / val 나누는 기준을 이미지에 대해서 random 이 아닌
        사람(profile)을 기준으로 나눕니다.
        구현은 val_ratio 에 맞게 train / val 나누는 것을 이미지 전체가 아닌 사람(profile)에 대해서 진행하여 indexing 을 합니다
        이후 `split_dataset` 에서 index 에 맞게 Subset 으로 dataset 을 분기합니다.
    """

    def __init__(self, data_dir, mean=(0.548, 0.504, 0.479), std=(0.237, 0.247, 0.246), val_ratio=0.2):
        self.indices = defaultdict(list)
        super().__init__(data_dir, mean, std, val_ratio)

    @staticmethod
    def _split_profile(profiles, val_ratio):
        length = len(profiles)
        n_val = int(length * val_ratio)

        val_indices = set(random.choices(range(length), k=n_val))
        train_indices = set(range(length)) - val_indices
        return {
            "train": train_indices,
            "val": val_indices
        }

    def setup(self):
        profiles = os.listdir(self.data_dir)
        profiles = [profile for profile in profiles if not profile.startswith(".")]
        split_profiles = self._split_profile(profiles, self.val_ratio)

        cnt = 0
        for phase, indices in split_profiles.items():
            for _idx in indices:
                profile = profiles[_idx]
                img_folder = os.path.join(self.data_dir, profile)
                for file_name in os.listdir(img_folder):
                    _file_name, ext = os.path.splitext(file_name)
                    if _file_name not in self._file_names:  # "." 로 시작하는 파일 및 invalid 한 파일들은 무시합니다
                        continue

                    img_path = os.path.join(self.data_dir, profile, file_name)  # (resized_data, 000004_male_Asian_54, mask1.jpg)
                    mask_label = self._file_names[_file_name]

                    id, gender, race, age = profile.split("_")
                    gender_label = GenderLabels.from_str(gender)
                    age_label = AgeLabels.from_number(age)

                    self.image_paths.append(img_path)
                    self.mask_labels.append(mask_label)
                    self.gender_labels.append(gender_label)
                    self.age_labels.append(age_label)

                    self.indices[phase].append(cnt)
                    cnt += 1

    def split_dataset(self) -> List[Subset]:
        return [Subset(self, indices) for phase, indices in self.indices.items()]


class TestDataset(Dataset):
    def __init__(self, img_paths, resize, mean=(0.548, 0.504, 0.479), std=(0.237, 0.247, 0.246)):
        self.img_paths = img_paths
        self.transform = Compose([
            Resize(resize, Image.BILINEAR),
            ToTensor(),
            Normalize(mean=mean, std=std),
        ])

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.img_paths)

In [6]:
base_dataset = MaskBaseDataset("/opt/ml/input/data/train/images")
base_dataset.set_transform(BaseAugmentation(resize=(128, 128), mean=(0.5, 0, 0), std=(1,1,1)))
train_set, val_set = base_dataset.split_dataset()

# train_loader = DataLoader(
#     train_set,
#     batch_size=1000
#     # num_workers=
#     shuffle=True,
#     pin_memory=use_cuda,
#     drop_last=True,
# )

val_loader = DataLoader(
    val_set,
    batch_size=100,
    #num_workers=multiprocessing.cpu_count()//2,
    shuffle=False,
    pin_memory=True,
    drop_last=True,
)

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class BaseModel(nn.Module):
    def __init__(self, num_classes):
        super().__init__()

        self.conv1 = nn.Conv2d(3, 32, kernel_size=7, stride=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.25)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)

        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)

        x = self.conv3(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout2(x)

        x = self.avgpool(x)
        x = x.view(-1, 128)
        return self.fc(x)

class ResNet18(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.resnet18 = models.resnet18(pretrained=True)
        self.resnet18.fc = nn.Linear(in_features=512, out_features = num_classes, bias= True) 
        # self.Linear_1 = nn.Linear(in_features=1000, out_features = num_classes)

        # initialize w & b
        torch.nn.init.xavier_uniform_(self.resnet18.fc.weight)
        stdv = 1/math.sqrt(self.resnet18.fc.in_features)
        self.resnet18.fc.bias.data.uniform_(-stdv, stdv)

    def forward(self, x):
        x = self.resnet18(x)
        return x

# EfficientNet
from efficientnet_pytorch import EfficientNet
import math
class EfficientNetB3(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.efficientnet = EfficientNet.from_pretrained('efficientnet-b3')
        # Drop out will be added Here
        self.efficientnet._fc = nn.Linear(in_features=1536, out_features = num_classes) 
        # self.linear = nn.Linear(in_features = self.efficientnet._fc.out_features, out_features = num_classes)

        # initialize w & b
        torch.nn.init.xavier_uniform_(self.efficientnet._fc.weight)
        stdv = 1/math.sqrt(self.efficientnet._fc.in_features)
        self.efficientnet._fc.bias.data.uniform_(-stdv, stdv)

    def forward(self, x):
        x = self.efficientnet(x)
        return x


# Custom Model Template
class MyModel(nn.Module):
    def __init__(self, num_classes):
        super().__init__()

        """
        1. 위와 같이 생성자의 parameter 에 num_claases 를 포함해주세요.
        2. 나만의 모델 아키텍쳐를 디자인 해봅니다.
        3. 모델의 output_dimension 은 num_classes 로 설정해주세요.
        """

    def forward(self, x):
        """
        1. 위에서 정의한 모델 아키텍쳐를 forward propagation 을 진행해주세요
        2. 결과로 나온 output 을 return 해주세요
        """
        return x


In [8]:
class EfficientNetB3(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.efficientnet = EfficientNet.from_pretrained('efficientnet-b3')
        # Drop out will be added Here
        in_features = self.efficientnet._fc.in_features
        self.efficientnet._fc = nn.Linear(in_features=in_features, out_features = num_classes) 
        # self.linear = nn.Linear(in_features = self.efficientnet._fc.out_features, out_features = num_classes)

        # initialize w & b
        torch.nn.init.xavier_uniform_(self.efficientnet._fc.weight)
        stdv = 1/math.sqrt(self.efficientnet._fc.in_features)
        self.efficientnet._fc.bias.data.uniform_(-stdv, stdv)

    def forward(self, x):
        x = self.efficientnet(x)
        return x


In [9]:
EffNet = EfficientNet.from_pretrained('efficientnet-b3')
EffNet
import torchsummary 
torchsummary.summary(EffNet, (3, 128, 128), device='cpu')

Loaded pretrained weights for efficientnet-b3
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         ZeroPad2d-1          [-1, 3, 129, 129]               0
Conv2dStaticSamePadding-2           [-1, 40, 64, 64]           1,080
       BatchNorm2d-3           [-1, 40, 64, 64]              80
MemoryEfficientSwish-4           [-1, 40, 64, 64]               0
         ZeroPad2d-5           [-1, 40, 66, 66]               0
Conv2dStaticSamePadding-6           [-1, 40, 64, 64]             360
       BatchNorm2d-7           [-1, 40, 64, 64]              80
MemoryEfficientSwish-8           [-1, 40, 64, 64]               0
          Identity-9             [-1, 40, 1, 1]               0
Conv2dStaticSamePadding-10             [-1, 10, 1, 1]             410
MemoryEfficientSwish-11             [-1, 10, 1, 1]               0
         Identity-12             [-1, 10, 1, 1]               0
Conv2dStaticSamePadding-13        

In [10]:
print(EffNet)

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 40, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_bn0): BatchNorm2d(40, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        40, 40, kernel_size=(3, 3), stride=[1, 1], groups=40, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): BatchNorm2d(40, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        40, 10, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        10, 40, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        40, 24, kernel_siz

In [11]:
from pytorch_pretrained_vit import ViT
model = ViT('B_16_imagenet1k', pretrained=True)

class ViTPretrainedFreeze(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.model = ViT('B_16_imagenet1k', pretrained=True)
        
        for param in self.model.parameters():
            param.requires_grad = False

        in_features = self.model.fc.in_features
        self.model.fc = nn.Linear(in_features=in_features, out_features= num_classes, bias= True)

        # initialize
        torch.nn.init.xavier_uniform_(self.model.fc.weight)
        stdv = 1/math.sqrt(self.model.fc.in_features)
        self.model.fc.bias.data.uniform_(-stdv, stdv)

    def forward(self, x):
        x = self.model(x)
        return x


Loaded pretrained weights.


In [12]:
model = ViT('B_16_imagenet1k', pretrained=True)

Loaded pretrained weights.


In [3]:
import torch
torch.cuda.empty_cache()

In [4]:
# !pip install GPUtil
import GPUtil
GPUtil.showUtilization()

| ID | GPU | MEM |
------------------
|  0 |  0% | 18% |


In [155]:

model = ViTPretrainedFreeze(18)
model(torch.Tensor(np.ones((1, 3, 244, 244)))).squeeze(0)


Loaded pretrained weights.


RuntimeError: The size of tensor a (226) must match the size of tensor b (577) at non-singleton dimension 1

In [144]:
model.eval()
model(torch.Tensor(np.ones((2, 3,384, 384)))).size()


torch.Size([2, 3, 384, 384])

In [130]:
torchsummary.summary(model, input_size=(3,384, 384), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 768, 24, 24]         590,592
PositionalEmbedding1D-2             [-1, 577, 768]               0
         LayerNorm-3             [-1, 577, 768]           1,536
            Linear-4             [-1, 577, 768]         590,592
            Linear-5             [-1, 577, 768]         590,592
            Linear-6             [-1, 577, 768]         590,592
           Dropout-7         [-1, 12, 577, 577]               0
MultiHeadedSelfAttention-8             [-1, 577, 768]               0
            Linear-9             [-1, 577, 768]         590,592
          Dropout-10             [-1, 577, 768]               0
        LayerNorm-11             [-1, 577, 768]           1,536
           Linear-12            [-1, 577, 3072]       2,362,368
           Linear-13             [-1, 577, 768]       2,360,064
PositionWiseFeedForward-14    

In [5]:
import albumentations as A
import cv2


In [9]:
from torchvision.transforms import *

In [12]:
transform = A.Compose([
    Pad((114,0,114,114)),
    CenterCrop((360, 360)),
    A.HorizontalFlip(p=.5),

    A.augmentations.geometric.transforms.ShiftScaleRotate(rotate_limit= 10, p = 0.2),
    A.augmentations.transforms.GaussNoise(var_limit=(10,20), mean=0, per_channel= True, p=0.4),
    A.augmentations.geometric.transforms.ShiftScaleRotate(),
    A.augmentations.transforms.CLAHE(),
    A.augmentations.transforms.RandomBrightnessContrast(),
    A.augmentations.transforms.RandomGamma(),
    A.augmentations.transforms.HueSaturationValue(),
    A.augmentations.transforms.RGBShift(),

    # Resize(resize, Image.BILINEAR),
    # ToTensor(),
    # Normalize(mean=mean, std=std),
])

In [15]:
import pandas as pd
from PIL import Image



train_dir = '/opt/ml/input/data/train'

train_info = pd.read_csv(os.path.join(train_dir, 'train.csv'))
image_dir = os.path.join(train_dir, 'images')

image_paths = [os.path.join(image_dir, img_id) for img_id in train_info.path]

example_image = Image.open(os.path.join(image_paths[500], 'mask5.jpg'))

In [19]:
transform(image = example_image)['image']

TypeError: image must be numpy array type

In [30]:
transform = A.Compose([
    A.Resize(*(2,2))
])
from albumentations.pytorch import ToTensorV2


In [31]:
class GoodAugmentation:
    """
    사람들이 많이 사용하는 Augmentation의 집합
    """
    def __init__(self, resize, mean, std, **args):

        self.transform = A.Compose([
            # Pad((114,0,114,114)),
            # CenterCrop((360, 360)),
            A.Resize(*resize), 
            A.HorizontalFlip(p=.5),
            A.augmentations.geometric.transforms.ShiftScaleRotate(rotate_limit= 10, p = 0.2),
            A.augmentations.transforms.GaussNoise(var_limit=(10,20), mean=0, per_channel= True, p=0.4),
            A.augmentations.geometric.transforms.ShiftScaleRotate(),
            A.augmentations.transforms.CLAHE(),
            A.augmentations.transforms.RandomBrightnessContrast(),
            A.augmentations.transforms.RandomGamma(),
            A.augmentations.transforms.HueSaturationValue(),
            A.augmentations.transforms.RGBShift(),
            A.Normalize(mean=mean, std=std),
            ToTensorV2()
        ])


    def __call__(self, image):
        return self.transform(image)

In [ ]:
for param in model.parameters():
            param.requires_grad = False

In [7]:
transform = A.Compose([
    A.RandomBrightnessContrast(brightness_limit=1, contrast_limit=1, p=1.0),
])
transformed_image_1 = transform(image=image)['image']
transformed_image_2 = transform(image=image)['image']
transformed_image_3 = transform(image=image)['image']


NameError: name 'A' is not defined

In [152]:
class ResNet18Dropout(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.resnet18 = models.resnet18(pretrained=True)
        self.resnet18.fc = nn.Linear(in_features=512, out_features=num_classes, bias=True)
        # https://discuss.pytorch.org/t/inject-dropout-into-resnet-or-any-other-network/66322/3
        self.resnet18.fc.register_forward_hook(lambda m, inp, out: F.dropout(out, p=0.5, training=m.training))        

        # initialize w & b
        torch.nn.init.xavier_uniform_(self.resnet18.fc.weight)
        stdv = 1 / math.sqrt(self.resnet18.fc.in_features)
        self.resnet18.fc.bias.data.uniform_(-stdv, stdv)

    def forward(self, x):
        x = self.resnet18(x)
        return x

In [153]:
model = ResNet18Dropout(18)
model

ResNet18Dropout(
  (resnet18): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, 

In [154]:
import torchsummary
torchsummary.summary(model, (3,299,299), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 150, 150]           9,408
       BatchNorm2d-2         [-1, 64, 150, 150]             128
              ReLU-3         [-1, 64, 150, 150]               0
         MaxPool2d-4           [-1, 64, 75, 75]               0
            Conv2d-5           [-1, 64, 75, 75]          36,864
       BatchNorm2d-6           [-1, 64, 75, 75]             128
              ReLU-7           [-1, 64, 75, 75]               0
            Conv2d-8           [-1, 64, 75, 75]          36,864
       BatchNorm2d-9           [-1, 64, 75, 75]             128
             ReLU-10           [-1, 64, 75, 75]               0
       BasicBlock-11           [-1, 64, 75, 75]               0
           Conv2d-12           [-1, 64, 75, 75]          36,864
      BatchNorm2d-13           [-1, 64, 75, 75]             128
             ReLU-14           [-1, 64,

In [48]:
model = BaseModel(18)
model.register_forward_hook(lambda m, inp, out: F.dropout(out, p=0.5, training=m.training))

In [59]:
resnet18 = models.resnet18(pretrained=True)
num_classes = 18
resnet18.fc = nn.Dropout(0.5)
nn.Linear(in_features=512, out_features=num_classes, bias=True)

import torchsummary
torchsummary.summary(resnet18, (3,111,111), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 56, 56]           9,408
       BatchNorm2d-2           [-1, 64, 56, 56]             128
              ReLU-3           [-1, 64, 56, 56]               0
         MaxPool2d-4           [-1, 64, 28, 28]               0
            Conv2d-5           [-1, 64, 28, 28]          36,864
       BatchNorm2d-6           [-1, 64, 28, 28]             128
              ReLU-7           [-1, 64, 28, 28]               0
            Conv2d-8           [-1, 64, 28, 28]          36,864
       BatchNorm2d-9           [-1, 64, 28, 28]             128
             ReLU-10           [-1, 64, 28, 28]               0
       BasicBlock-11           [-1, 64, 28, 28]               0
           Conv2d-12           [-1, 64, 28, 28]          36,864
      BatchNorm2d-13           [-1, 64, 28, 28]             128
             ReLU-14           [-1, 64,

In [57]:
resnet18 = models.resnet18(pretrained=True)
num_classes = 18
resnet18.fc = nn.ModuleList(
    [nn.Dropout(0.5),
    nn.Linear(in_features=512, out_features=num_classes, bias=True)]
)



In [134]:
class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.one = nn.Linear(in_features=2, out_features=3, bias=False)
        self.one.weight.requires_grad = False
        self.one.register_forward_hook(lambda m, inp, out: F.dropout(out, p=0.5, training=m.training))

    def forward(self, x):
        x  = self.one(x)
        return x

In [135]:
model = MyModel()

In [150]:
model(torch.ones(1,2))

tensor([[ 0.0000, -0.2037, -1.7893]])

In [151]:
model

MyModel(
  (one): Linear(in_features=2, out_features=3, bias=False)
)

In [26]:

resnet18.fc.register_forward_hook(lambda m, inp: F.dropout(input=inp, p=0.5, training=m.training))

In [25]:
resnet18.avgpool.output_size

(1, 1)

In [4]:
resnet18 = models.resnet18(pretrained=True)
resnet18

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [6]:
class MyModel(nn.Module):
    def __init__(self, classes):
        super().__init__()
        self.out_dim = classes
        self.resnet = models.resnet18(pretrained=True)

        self.dropouts = nn.ModuleList([
            nn.Dropout(0.5) for _ in range(5)
        ])


        self.linear = nn.Linear(in_features=2, out_features=3, bias=False)
        self.one.weight.requires_grad = False
        self.one.register_forward_hook(lambda m, inp, out: F.dropout(out, p=0.5, training=m.training))

    def forward(self, x):
        x  = self.one(x)
        return x

import torch.nn as nn
class ResNet18MSD(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.resnet18 = models.resnet18(pretrained=True)
        self.dropouts = nn.ModuleList([
            nn.Dropout(0.5) for _ in range(5)
        ])
        self.linear = nn.Linear(in_features=2, out_features=num_classes, bias=False)

        # initialize w & b
        torch.nn.init.xavier_uniform_(self.resnet18.fc.weight)
        stdv = 1 / math.sqrt(self.resnet18.fc.in_features)
        self.resnet18.fc.bias.data.uniform_(-stdv, stdv)

    def forward(self, x):
        x = self.resnet18(x)
        for i, dropout in enumerate(self.dropouts):
            if i == 0:
                h = self.linear(dropout(x))
            else:
                h += self.linear(dropout(x))
        output = h / len(self.dropouts)
        return output

NameError: name 'nn' is not defined

In [67]:
import torch.nn as nn
class ResNet18MSD(nn.Module):
    """_summary_
    Name: Multi-sample Dropout ResNet18-pretrained

    Args:
        nn (_type_): _description_
    
    self.dropout_p = 0.5 (default):  
        the percentile of each dropout layers
        
    self.dropout_n = 5 (default):  
        the number of dropout layers


    ref: https://www.kaggle.com/c/jigsaw-unintended-bias-in-toxicity-classification/discussion/100961
    """
    def __init__(self, num_classes):
        super().__init__()
        self.dropout_p = 0.5
        self.dropout_n = 5
        self.resnet18 = models.resnet18(pretrained=True)
        self.dropouts = nn.ModuleList([
            nn.Dropout(self.dropout_p) for _ in range(self.dropout_n)
        ])
        in_features = self.resnet18.fc.out_features
        self.linear = nn.Linear(in_features=in_features, out_features=num_classes, bias=True)

        # initialize w & b
        torch.nn.init.xavier_uniform_(self.resnet18.fc.weight)
        stdv = 1 / math.sqrt(self.resnet18.fc.in_features)
        self.resnet18.fc.bias.data.uniform_(-stdv, stdv)

    def forward(self, x):
        x = self.resnet18(x)
        for i, dropout in enumerate(self.dropouts):
            if i == 0:
                h = self.linear(dropout(x))
            else:
                h += self.linear(dropout(x))
        output = h / len(self.dropouts)
        return output

In [58]:
dropout_p = 0.5
dropout_n = 5
num_classes = 18

resnet18 = models.resnet18(pretrained=True)
dropouts = nn.ModuleList([
    nn.Dropout(dropout_p) for _ in range(dropout_n)
])
in_features = resnet18.fc.out_features
linear = nn.Linear(in_features=in_features, out_features=num_classes, bias=True)

In [59]:
data = torch.zeros(1, 3,256, 256)
x = resnet18(data)



In [60]:
print(x.size())
print(dropouts[0](x).size())

torch.Size([1, 1000])
torch.Size([1, 1000])


In [61]:
linear(dropouts[0](x).flatten())

tensor([-0.5756, -0.9549, -1.4656, -0.3327,  0.9887, -1.3598, -0.3762,  0.4157,
         0.5999,  0.8185,  1.9030,  0.2581,  1.1759, -0.3088,  1.5117, -0.0159,
         0.5780, -0.0336], grad_fn=<AddBackward0>)

In [62]:
linear(dropouts[0](x))

tensor([[ 0.1287,  0.1866, -1.1070,  1.0414, -0.2110, -0.3907, -0.4316,  0.2452,
          0.7150,  1.1592,  1.0270, -0.8453,  0.4880, -0.4587,  1.3571,  0.1753,
          0.5610,  0.0103]], grad_fn=<AddmmBackward>)

In [42]:
dropouts[0](x).size() + dropouts[1](x).size()

torch.Size([1, 1000, 1, 1000])

In [69]:
import math
model = ResNet18MSD(18)

In [70]:
import torchsummary
torchsummary.summary(model, (3, 111,111), device='cpu')



----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 56, 56]           9,408
       BatchNorm2d-2           [-1, 64, 56, 56]             128
              ReLU-3           [-1, 64, 56, 56]               0
         MaxPool2d-4           [-1, 64, 28, 28]               0
            Conv2d-5           [-1, 64, 28, 28]          36,864
       BatchNorm2d-6           [-1, 64, 28, 28]             128
              ReLU-7           [-1, 64, 28, 28]               0
            Conv2d-8           [-1, 64, 28, 28]          36,864
       BatchNorm2d-9           [-1, 64, 28, 28]             128
             ReLU-10           [-1, 64, 28, 28]               0
       BasicBlock-11           [-1, 64, 28, 28]               0
           Conv2d-12           [-1, 64, 28, 28]          36,864
      BatchNorm2d-13           [-1, 64, 28, 28]             128
             ReLU-14           [-1, 64,

In [142]:
class ResNet18(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.resnet18 = models.resnet18(pretrained=True)
        self.resnet18.fc = nn.Linear(in_features=512, out_features=num_classes, bias=True)
        # self.Linear_1 = nn.Linear(in_features=1000, out_features = num_classes)

        # initialize w & b
        torch.nn.init.xavier_uniform_(self.resnet18.fc.weight)
        stdv = 1 / math.sqrt(self.resnet18.fc.in_features)
        self.resnet18.fc.bias.data.uniform_(-stdv, stdv)

    def forward(self, x):
        x = self.resnet18(x)
        return x

In [8]:
class EfficientNetB4FreezeTop6(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.efficientnet = models.efficientnet_b4(pretrained=True)

        in_features = self.efficientnet.classifier[1].in_features
        self.efficientnet.classifier[1] = nn.Linear(in_features=in_features, out_features=num_classes)
        # self.linear = nn.Linear(in_features = self.efficientnet._fc.out_features, out_features = num_classes)

        # initialize w & b
        torch.nn.init.xavier_uniform_(self.efficientnet.classifier[1].weight)
        stdv = 1 / math.sqrt(self.efficientnet.classifier[1].in_features)
        self.efficientnet.classifier[1].bias.data.uniform_(-stdv, stdv)

        # Freeze Top 6 layers
        for i, child in enumerate(self.efficientnet.children()):
            if i == 6 : break # break point; Top 6 layers
            for param in child.parameters():
                param.requires_grad = False

    def forward(self, x):
        self.efficientnet(x)
        return

In [9]:
model = ResNet18(18)
torchsummary.summary(model, (3, 10,10), device='cpu')
model(torch.zeros(1,3,10,10))


/opt/ml/.venv/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1             [-1, 64, 5, 5]           9,408
       BatchNorm2d-2             [-1, 64, 5, 5]             128
              ReLU-3             [-1, 64, 5, 5]               0
         MaxPool2d-4             [-1, 64, 3, 3]               0
            Conv2d-5             [-1, 64, 3, 3]          36,864
       BatchNorm2d-6             [-1, 64, 3, 3]             128
              ReLU-7             [-1, 64, 3, 3]               0
            Conv2d-8             [-1, 64, 3, 3]          36,864
       BatchNorm2d-9             [-1, 64, 3, 3]             128
             ReLU-10             [-1, 64, 3, 3]               0
       BasicBlock-11             [-1, 64, 3, 3]               0
           Conv2d-12             [-1, 64, 3, 3]          36,864
      BatchNorm2d-13             [-1, 64, 3, 3]             128
             ReLU-14             [-1, 6

ValueError: Expected more than 1 value per channel when training, got input size torch.Size([1, 256, 1, 1])

In [90]:
efficientnet = models.efficientnet_b4(pretrained=True)
efficientnet.classifier[1] = nn.Linear(in_features=1792, out_features=18, bias=True)
efficientnet

EfficientNet(
  (features): Sequential(
    (0): ConvNormActivation(
      (0): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): ConvNormActivation(
            (0): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=48, bias=False)
            (1): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(48, 12, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(12, 48, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): ConvNormActivation(

In [20]:
import torchvision.models
class EfficientNetB3MSD(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.dropout_p = 0.5
        self.dropout_n = 5
        self.efficientnet = models.efficientnet_b3(pretrained=True)

        self.dropouts = nn.ModuleList([
            nn.Dropout(self.dropout_p) for _ in range(self.dropout_n)
        ])
        
        in_features = self.efficientnet.classifier[1].out_features
        self.linear = nn.Linear(in_features=in_features, out_features=num_classes)

        # initialize w & b
        torch.nn.init.xavier_uniform_(self.linear.weight)
        stdv = 1 / math.sqrt(self.linear.in_features)
        self.linear.bias.data.uniform_(-stdv, stdv)

    def forward(self, x):
        x = self.efficientnet(x)
        for i, dropout in enumerate(self.dropouts):
            if i == 0:
                h = self.linear(dropout(x))
            else:
                h += self.linear(dropout(x))
        output = h / len(self.dropouts)
        return output

In [21]:
import math
import torchsummary
model = EfficientNetB3MSD(18)
torchsummary.summary(model, (3, 111,111), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 40, 56, 56]           1,080
       BatchNorm2d-2           [-1, 40, 56, 56]              80
              SiLU-3           [-1, 40, 56, 56]               0
            Conv2d-4           [-1, 40, 56, 56]             360
       BatchNorm2d-5           [-1, 40, 56, 56]              80
              SiLU-6           [-1, 40, 56, 56]               0
 AdaptiveAvgPool2d-7             [-1, 40, 1, 1]               0
            Conv2d-8             [-1, 10, 1, 1]             410
              SiLU-9             [-1, 10, 1, 1]               0
           Conv2d-10             [-1, 40, 1, 1]             440
          Sigmoid-11             [-1, 40, 1, 1]               0
SqueezeExcitation-12           [-1, 40, 56, 56]               0
           Conv2d-13           [-1, 24, 56, 56]             960
      BatchNorm2d-14           [-1, 24,

In [33]:
model.eval()
model(torch.zeros(1,3,300,300))

tensor([[-1.7140,  0.6061, -2.1051, -3.1328,  0.8486, -2.2471, -1.0051, -0.7541,
         -0.4332,  1.0416,  0.4228, -0.6889,  1.4631, -2.1437, -0.5683,  1.4205,
         -0.2709,  0.5670]], grad_fn=<DivBackward0>)

In [13]:
model = models.efficientnet_b3(pretrained=True)
model.classifier[1].out_features

model.eval()
model(torch.zeros(1,3,10,10))

1000

In [37]:
model = models.efficientnet_b7(pretrained=True)
model.classifier[1].out_features

model.eval()
model(torch.zeros(1,3,10,10))

Downloading: "https://download.pytorch.org/models/efficientnet_b7_lukemelas-dcc49843.pth" to /opt/ml/.cache/torch/hub/checkpoints/efficientnet_b7_lukemelas-dcc49843.pth
100.0%


tensor([[ 3.0540e+00,  2.0709e+01, -2.9925e+00, -1.3394e+01, -4.3679e+00,
          8.7981e+00,  1.5670e+00,  1.1556e+01,  5.5424e+00,  1.4670e+01,
          1.5921e+01,  9.0159e+00,  2.2124e+01,  1.2676e+01,  1.7205e+01,
          1.6620e+01,  1.6054e+01,  1.0669e+01,  1.1039e+01,  1.6499e+01,
          1.3581e+01,  1.5102e+01,  1.6478e+01,  1.4321e+01,  9.7658e+00,
          5.8394e+00, -1.2077e+00,  7.3977e-01,  8.2200e+00,  1.8875e+01,
          1.6999e+01,  8.9023e+00,  8.9871e+00,  1.0031e+01,  1.6364e+00,
          1.3340e+01,  2.3333e+01,  1.7458e+01,  4.1186e+00,  1.5860e+01,
          1.1535e+00,  7.3617e+00,  1.1950e+01,  1.4593e+01, -3.9760e-01,
          1.2964e+01,  1.7996e-01,  1.6309e+01,  1.5015e+01,  7.8031e+00,
          9.1965e+00,  2.3562e+01, -2.1839e-01, -2.8472e+00, -9.2983e-01,
          5.2668e+00, -5.4055e+00,  3.4783e+00,  3.5437e+00,  1.3067e+01,
          5.2817e-01,  1.8916e+01,  1.5462e+01,  1.2782e+01,  1.1093e+01,
         -3.8048e+00,  1.2126e+01,  1.

In [11]:
model = torchvision.models.efficientnet_b3(pretrained=True)
model

Downloading: "https://download.pytorch.org/models/efficientnet_b3_rwightman-cf984f9c.pth" to /opt/ml/.cache/torch/hub/checkpoints/efficientnet_b3_rwightman-cf984f9c.pth
100.0%


EfficientNet(
  (features): Sequential(
    (0): ConvNormActivation(
      (0): Conv2d(3, 40, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): ConvNormActivation(
            (0): Conv2d(40, 40, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=40, bias=False)
            (1): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(40, 10, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(10, 40, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): ConvNormActivation(

In [7]:
import torchvision

In [8]:
import math
model = EfficientNetB4MSD(18)

import torchsummary
torchsummary.summary(model, (3, 111,111), device='cpu')

AttributeError: 'EfficientNet' object has no attribute 'fc'

In [13]:
import torchvision
torchvision.__version__

'0.8.2'

In [36]:
import ttach as tta

transforms = tta.Compose(
    [
        tta.HorizontalFlip(),
        tta.Multiply(factors=[0.9, 1, 1.1]),        
    ]
)
tta_model = tta.ClassificationTTAWrapper(model, transforms)



In [28]:
#### 03.01
class ResNet34FreezeTop6(nn.Module):
    """
    """
    def __init__(self, num_classes):
        super().__init__()
        self.model = models.resnet34(pretrained=True)
        self.model.fc = nn.Linear(in_features=self.model.fc.in_features, out_features=num_classes, bias=True)

        # initialize w & b
        torch.nn.init.xavier_uniform_(self.model.fc.weight)
        stdv = 1 / math.sqrt(self.model.fc.in_features)
        self.model.fc.bias.data.uniform_(-stdv, stdv)

        # freeze top 6 layers
        for i, child in enumerate(self.model.children()):
            if i == 6 : break # break point
            for param in child.parameters():
                param.requires_grad = False

    def forward(self, x):
        x = self.model(x)
        return x



In [36]:
model = ResNet34FreezeTop6(18)

In [37]:
model

ResNet34FreezeTop6(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, 

In [38]:

torchsummary.summary(model, (3, 111,111), device='cpu')


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 56, 56]           9,408
       BatchNorm2d-2           [-1, 64, 56, 56]             128
              ReLU-3           [-1, 64, 56, 56]               0
         MaxPool2d-4           [-1, 64, 28, 28]               0
            Conv2d-5           [-1, 64, 28, 28]          36,864
       BatchNorm2d-6           [-1, 64, 28, 28]             128
              ReLU-7           [-1, 64, 28, 28]               0
            Conv2d-8           [-1, 64, 28, 28]          36,864
       BatchNorm2d-9           [-1, 64, 28, 28]             128
             ReLU-10           [-1, 64, 28, 28]               0
       BasicBlock-11           [-1, 64, 28, 28]               0
           Conv2d-12           [-1, 64, 28, 28]          36,864
      BatchNorm2d-13           [-1, 64, 28, 28]             128
             ReLU-14           [-1, 64,

In [39]:
# model = models.resnet34(pretrained=True)
model.eval()
model(torch.zeros(1, 3,10,10)).size()


torch.Size([1, 18])

In [117]:
import torchsummary

model = ResNet34FreezeTop6(18)
torchsummary.summary(model, (3, 111,111), device='cpu')
model(torch.zeros(1, 3,10,10))
model

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 56, 56]           9,408
       BatchNorm2d-2           [-1, 64, 56, 56]             128
              ReLU-3           [-1, 64, 56, 56]               0
         MaxPool2d-4           [-1, 64, 28, 28]               0
            Conv2d-5           [-1, 64, 28, 28]          36,864
       BatchNorm2d-6           [-1, 64, 28, 28]             128
              ReLU-7           [-1, 64, 28, 28]               0
            Conv2d-8           [-1, 64, 28, 28]          36,864
       BatchNorm2d-9           [-1, 64, 28, 28]             128
             ReLU-10           [-1, 64, 28, 28]               0
       BasicBlock-11           [-1, 64, 28, 28]               0
           Conv2d-12           [-1, 64, 28, 28]          36,864
      BatchNorm2d-13           [-1, 64, 28, 28]             128
             ReLU-14           [-1, 64,

ValueError: Expected more than 1 value per channel when training, got input size torch.Size([1, 256, 1, 1])

In [91]:
model1 = models.resnet34(pretrained=True)
print()
torchsummary.summary(model1, (3, 111,111), device='cpu')

TypeError: object of type 'ResNet' has no len()

In [70]:
model1.fc = nn.Linear(512, 18)

In [75]:
model = models.efficientnet_b3(pretrained=True)
model.train()



Linear(in_features=512, out_features=1000, bias=True)

In [56]:
model.eval()
model

EfficientNet(
  (features): Sequential(
    (0): ConvNormActivation(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): ConvNormActivation(
            (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
            (1): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(64, 16, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): ConvNormActivatio

In [55]:
model.eval()
model(torch.zeros(1,3,10,10))

In [81]:
for i in model.children():
    print(i)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [1]:
import GPUtil

In [38]:
class EfficientNetB7(nn.Module):
    """_summary_
    size recommended to 600
    """
    def __init__(self, num_classes):
        super().__init__()
        self.efficientnet = models.efficientnet_b7(pretrained=True)

        in_features = self.efficientnet.classifier[1].in_features
        self.efficientnet.classifier[1] = nn.Linear(in_features=in_features, out_features=num_classes)
        # self.linear = nn.Linear(in_features = self.efficientnet._fc.out_features, out_features = num_classes)

        # initialize w & b
        torch.nn.init.xavier_uniform_(self.efficientnet.classifier[1].weight)
        stdv = 1 / math.sqrt(self.efficientnet.classifier[1].in_features)
        self.efficientnet.classifier[1].bias.data.uniform_(-stdv, stdv)

    def forward(self, x):
        x =  self.efficientnet(x)
        return x

In [64]:
class EfficientNetB3(nn.Module):
    """_summary_
    size recommended to 300
    """
    def __init__(self, num_classes):
        super().__init__()
        self.efficientnet = models.efficientnet_b3(pretrained=True)

        in_features = self.efficientnet.classifier[1].in_features
        self.efficientnet.classifier[1] = nn.Linear(in_features=in_features, out_features=num_classes)
        # self.linear = nn.Linear(in_features = self.efficientnet._fc.out_features, out_features = num_classes)

        # initialize w & b
        torch.nn.init.xavier_uniform_(self.efficientnet.classifier[1].weight)
        stdv = 1 / math.sqrt(self.efficientnet.classifier[1].in_features)
        self.efficientnet.classifier[1].bias.data.uniform_(-stdv, stdv)

    def forward(self, x):
        x =  self.efficientnet(x)
        return x


model = EfficientNetB3(18) 

# model_path = os.path.join(saved_model, args.state+'best.pth' ) # default : best.pth
model_path = '/opt/ml/level1-image-classification-level1-recsys-11/BaseLineCodeV2/model/general/LableSmoothEfficientNetB3_Kfold/best_4.pth'
model.load_state_dict(torch.load(model_path, map_location=torch.device("cuda")))

model.eval()
# outs = model(torch.zeros(2,3,10,10))
# outs

EfficientNetB3(
  (efficientnet): EfficientNet(
    (features): Sequential(
      (0): ConvNormActivation(
        (0): Conv2d(3, 40, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): SiLU(inplace=True)
      )
      (1): Sequential(
        (0): MBConv(
          (block): Sequential(
            (0): ConvNormActivation(
              (0): Conv2d(40, 40, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=40, bias=False)
              (1): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): SiLU(inplace=True)
            )
            (1): SqueezeExcitation(
              (avgpool): AdaptiveAvgPool2d(output_size=1)
              (fc1): Conv2d(40, 10, kernel_size=(1, 1), stride=(1, 1))
              (fc2): Conv2d(10, 40, kernel_size=(1, 1), stride=(1, 1))
              (activation): SiLU(inplace=True)
              (s

In [66]:
model(torch.zeros(2,3,10,10)) + model(torch.zeros(2,3,10,10))

tensor([[ 0.9414,  0.8188,  1.6813, -0.1791,  0.6231,  1.5483,  1.1100,  0.8472,
          2.7434,  0.8915,  0.6891,  2.1064,  1.5155,  2.2162,  2.7740,  0.1257,
          1.5026,  2.3525],
        [ 0.9414,  0.8188,  1.6813, -0.1791,  0.6231,  1.5483,  1.1100,  0.8472,
          2.7434,  0.8915,  0.6891,  2.1064,  1.5155,  2.2162,  2.7740,  0.1257,
          1.5026,  2.3525]], grad_fn=<AddBackward0>)

In [62]:
# model_path = os.path.join(saved_model, args.state+'best.pth' ) # default : best.pth
model_path = '/opt/ml/level1-image-classification-level1-recsys-11/BaseLineCodeV2/model/general/LableSmoothEfficientNetB3_Kfold/best_4.pth'
model.load_state_dict(torch.load(model_path, map_location=torch.device("cuda")))

RuntimeError: Error(s) in loading state_dict for EfficientNetB7:
	Missing key(s) in state_dict: "efficientnet.features.1.2.block.0.0.weight", "efficientnet.features.1.2.block.0.1.weight", "efficientnet.features.1.2.block.0.1.bias", "efficientnet.features.1.2.block.0.1.running_mean", "efficientnet.features.1.2.block.0.1.running_var", "efficientnet.features.1.2.block.1.fc1.weight", "efficientnet.features.1.2.block.1.fc1.bias", "efficientnet.features.1.2.block.1.fc2.weight", "efficientnet.features.1.2.block.1.fc2.bias", "efficientnet.features.1.2.block.2.0.weight", "efficientnet.features.1.2.block.2.1.weight", "efficientnet.features.1.2.block.2.1.bias", "efficientnet.features.1.2.block.2.1.running_mean", "efficientnet.features.1.2.block.2.1.running_var", "efficientnet.features.1.3.block.0.0.weight", "efficientnet.features.1.3.block.0.1.weight", "efficientnet.features.1.3.block.0.1.bias", "efficientnet.features.1.3.block.0.1.running_mean", "efficientnet.features.1.3.block.0.1.running_var",

In [ ]:
model

In [ ]:
torch.argmax(outs, dim=-1)

In [ ]:
model = models.efficientnet_b3(pretrained=True)
model.classifier[1].out_features

model.eval()
model(torch.zeros(1,3,10,10))

1000

In [40]:
import argparse
import numpy as np
import pandas as pd
import os

def concat_outputs_voting(args):
    for i in range(5): # K-fold
        output_df = pd.read_csv(os.path.join(args.output_dir, f'output{i}'.csv))
        output_df['ans']
        if i == 0:
            submission = pd.DataFrame()
            submission['ImageID']
            submission[0] = output_df['ans']
        else :
            submission[i] = output_df['ans']    
    
    submission.set_index('ImageID', inplace=True)

In [47]:
for i in range(5): # K-fold
        output_df = pd.read_csv(os.path.join('./output/Kfold-test-0', f'output{i}.csv'))
        output_df['ans']
        if i == 0:
            submission = pd.DataFrame()
            submission['ImageID'] = output_df['ImageID']
            submission[0] = output_df['ans']
        else :
            submission[i] = output_df['ans']

In [ ]:
model()

In [55]:
submission.set_index('ImageID').mode(axis=1)

,0,1
ImageID,,
cbc5c6e168e63498590db46022617123f1fe1268.jpg,13.0,NaN
0e72482bf56b3581c081f7da2a6180b8792c7089.jpg,2.0,4.0
b549040c49190cedc41327748aeb197c1670f14d.jpg,16.0,NaN
4f9cb2a045c6d5b9e50ad3459ea7b791eb6e18bc.jpg,16.0,NaN
248428d9a4a5b6229a7081c32851b90cb8d38d0c.jpg,12.0,NaN
...,...,...
d71d4570505d6af8f777690e63edfa8d85ea4476.jpg,1.0,NaN
6cf1300e8e218716728d5820c0bab553306c2cfd.jpg,4.0,NaN
8140edbba31c3a824e817e6d5fb95343199e2387.jpg,9.0,NaN


In [ ]:
saved_model = '.model/general/LableSmoothEfficientNetB3_Kfold'
model_paths = []

model_cls = getattr(import_module("model"), args.model)
    model = model_cls(
        num_classes=num_classes
    )

for i in range(5):
    model_path = os.path.join(saved_model, args.state+f'_{i}.pth' ) # default : best.pth
    model_paths.append(model_path)


import argparse
import multiprocessing
import os
from importlib import import_module

import pandas as pd
import torch
from torch.utils.data import DataLoader

from dataset import TestDataset, MaskBaseDataset
import dataset
import albumentations as A
import ttach as tta

args = argparse

def load_model(saved_model, num_classes, device):
    model_cls = getattr(import_module("model"), args.model)
    models = []
    for i in range(5):
        model = model_cls(
            num_classes=num_classes
        )
        model_path = os.path.join(saved_model, args.state+f'_{i}.pth' ) # default : best.pth
        model.load_state_dict(torch.load(model_path, map_location=device))

        args.tta = False
        if args.tta == True : 
            print("TTA will be applied !")
            my_tta_transforms = tta.Compose([
                    tta.HorizontalFlip()
                ])
            model = tta.ClassificationTTAWrapper(model, my_tta_transforms)
        else :
            print("No TTA !")
        
        models.append(model)
    
    return models


@torch.no_grad()
def inference(data_dir, model_dir, output_dir, args):
    """
    """
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    num_classes = MaskBaseDataset.num_classes  # 18
    models = load_model(model_dir, num_classes, device).to(device)
    

    img_root = os.path.join(data_dir, 'images')
    info_path = os.path.join(data_dir, 'info.csv')
    info = pd.read_csv(info_path)

    img_paths = [os.path.join(img_root, img_id) for img_id in info.ImageID]


    # -- dataset
    dataset_module = getattr(import_module("dataset"), args.dataset)  # default: TestDataset
    dataset = dataset_module(img_paths, args.resize)
    # dataset = TestDataset(img_paths, args.resize)
    loader = torch.utils.data.DataLoader(
        dataset,
        batch_size=args.batch_size,
        num_workers=multiprocessing.cpu_count() // 2,
        shuffle=False,
        pin_memory=use_cuda,
        drop_last=False,
    )

    print("Calculating inference results..")
    preds = []
    with torch.no_grad():
        for idx, images in enumerate(loader):
            images = images.to(device)
            
            for model_idx, model in enumerate(models):
                if model_idx == 0:
                    pred = model(images)
                else :
                    pred += model(images)
            # pred = model(images)
            pred = pred.argmax(dim=-1)
            preds.extend(pred.cpu().numpy())

    info['ans'] = preds
    save_path = os.path.join(output_dir, f'output.csv')
    info.to_csv(save_path, index=False)
    print(f"Inference Done! Inference result saved at {save_path}")


def inference_by_single_models(which_model, data_dir, model_dir, output_dir, args):
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    # which_model should be 'mask' or 'genderAge'
    model_class = None
    if which_model == 'mask':
        num_classes = 3
        # model_class = 'MaskSplitByProfileDatasetForAlbumOnlyMask'
    elif which_model == 'genderAge':
        num_classes = 6
        # model_class = 'MaskSplitByProfileDatasetForAlbumOnlyGenderAge'
    else:
        assert 'no match model type'

    model = load_model(model_dir, num_classes, device).to(device)
    model.eval() 

    img_root = os.path.join(data_dir, 'images')
    info_path = os.path.join(data_dir, 'info.csv')
    info = pd.read_csv(info_path)

    img_paths = [os.path.join(img_root, img_id) for img_id in info.ImageID]


    # -- dataset
    dataset_module = getattr(import_module("dataset"), args.dataset)  # default: TestDataset
    dataset = dataset_module(img_paths, args.resize)
    # dataset = TestDataset(img_paths, args.resize)
    loader = torch.utils.data.DataLoader(
        dataset,
        batch_size=args.batch_size,
        num_workers=multiprocessing.cpu_count() // 2,
        shuffle=False,
        pin_memory=use_cuda,
        drop_last=False,
    )

    print("Calculating inference " + which_model +  " inference results..")
    preds = []
    with torch.no_grad():
        for idx, images in enumerate(loader):
            images = images.to(device)
            pred = model(images)
            pred = pred.argmax(dim=-1)
            preds.extend(pred.cpu().numpy())

    info['ans'] = preds
    save_path = os.path.join(output_dir, f'output_' + which_model + '.csv')
    info.to_csv(save_path, index=False)
    print('Inference ' + which_model + ' complete saved as ' + save_path)

def ensemble_row(mask, genderAge):
    mask_str = str(mask)
    genderAge_str = str(genderAge)

    mask_genderAge_str = mask_str + genderAge_str

    mask_genderAge_dict = {
        '00' : 0,
        '01' : 1,
        '02' : 2,
        '03' : 3,
        '04' : 4,
        '05' : 5,
        '10' : 6,
        '11' : 7,
        '12' : 8,
        '13' : 9,
        '14' : 10,
        '15' : 11,
        '20' : 12,
        '21' : 13,
        '22' : 14,
        '23' : 15,
        '24' : 16,
        '25' : 17,
    }

    return mask_genderAge_dict[mask_genderAge_str]




    


    # tarpath = os.path.join(saved_model, 'best.tar.gz')
    # tar = tarfile.open(tarpath, 'r:gz')
    # tar.extractall(path=saved_model)

    model_path = os.path.join(saved_model, args.state+'.pth' ) # default : best.pth
    model.load_state_dict(torch.load(model_path, map_location=device))

print("TTA will be applied !")
        my_tta_transforms = tta.Compose([
                tta.HorizontalFlip()
            ])
        model = tta.ClassificationTTAWrapper(model, my_tta_transforms)
    else :

